In [169]:
import requests
import numpy as np
from tqdm import tqdm_notebook
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [170]:
url_of_pages = []
# This is the link seaxhing neurodegenerative disease with most recent filter. Each page has 10 links to papers, so it is searching for 10 pages
URLs = ['https://pubmed.ncbi.nlm.nih.gov/?term=neurodegenerative+disease&filter=simsearch1.fha&filter=datesearch.y_1&sort=date&page='+str(i+1) for i in range(10)]
# r1 = requests.get('https://pubmed.ncbi.nlm.nih.gov/?term=neurodegenerative+disease&filter=simsearch1.fha')
for url in tqdm_notebook(URLs):
    temp = 0
    r1 = requests.get(url)
    z = r1.text[:]
    while temp == 0:
        try:
            a = z.index('value="http')
            z = z[a+7 : ]
            b = z.index('"\n')
            link = z[0:b]
            if link not in url_of_pages:
                url_of_pages.append(link)
        except:
            temp = 1              
url_of_pages

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


['https://pubmed.ncbi.nlm.nih.gov/33047398/',
 'https://pubmed.ncbi.nlm.nih.gov/33047393/',
 'https://pubmed.ncbi.nlm.nih.gov/33046606/',
 'https://pubmed.ncbi.nlm.nih.gov/33046555/',
 'https://pubmed.ncbi.nlm.nih.gov/33046546/',
 'https://pubmed.ncbi.nlm.nih.gov/33046512/',
 'https://pubmed.ncbi.nlm.nih.gov/33046475/',
 'https://pubmed.ncbi.nlm.nih.gov/33046245/',
 'https://pubmed.ncbi.nlm.nih.gov/33045869/',
 'https://pubmed.ncbi.nlm.nih.gov/33045815/',
 'https://pubmed.ncbi.nlm.nih.gov/33045309/',
 'https://pubmed.ncbi.nlm.nih.gov/33045253/',
 'https://pubmed.ncbi.nlm.nih.gov/33045243/',
 'https://pubmed.ncbi.nlm.nih.gov/33045230/',
 'https://pubmed.ncbi.nlm.nih.gov/33045106/',
 'https://pubmed.ncbi.nlm.nih.gov/33044818/',
 'https://pubmed.ncbi.nlm.nih.gov/33044808/',
 'https://pubmed.ncbi.nlm.nih.gov/33044802/',
 'https://pubmed.ncbi.nlm.nih.gov/33044454/',
 'https://pubmed.ncbi.nlm.nih.gov/33044443/',
 'https://pubmed.ncbi.nlm.nih.gov/33044192/',
 'https://pubmed.ncbi.nlm.nih.gov/

In [171]:
def give_abstract(link):
    '''
    link is basically the link for the webpage
    '''
    page = requests.get(link).text[:]
    a = page.find('id="enc-abstract">\n              \n                \n\n\n  <p>\n')
    temp = page[a+18:]
    b = temp.find('<div class="similar-articles" id="similar">')
    abstract = temp[0:b]
    abstract = abstract.replace('\n','')
    abstract = abstract.replace('<p>','')
    abstract = abstract.replace('</p>','')
    abstract = abstract.replace('</div>','')
    abstract = abstract.replace('  ','')
    abstract = abstract.replace('<strong class="sub-title">','')
    abstract = abstract.replace('</strong>','')
    if '<p class="copyright' in abstract:
        ix = abstract.index('<p class="copyright')
        abstract = abstract[:ix]
    if '<div' in abstract:
        ix = abstract.index('<div')
        abstract = abstract[:ix]
    if 'Keywords:' in abstract:
        ix = abstract.index('Keywords:')
        abstract = abstract[:ix]
    abstract = abstract.replace('/','')
    abstract = abstract.replace('<sup>','')
    abstract = abstract.replace('</sup>','')
    abstract = abstract.replace('<i>','')
    abstract = abstract.replace('</i>','')
    return abstract

In [ ]:
# Array of all abstracts
array_of_all_abstracts = []
for link in tqdm_notebook(url_of_pages):
    array_of_all_abstracts.append(give_abstract(link))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [168]:
#printing first 10 abstracts
array_of_all_abstracts[:10]

["Rationale:Alzheimer's disease (AD) is a chronic, severe, progressive neurodegenerative disorder associated with cognitive and memory impairment, and it ultimately causes death. Most approved drugs can only alleviate some of the symptoms of AD, but no interventions have been found that reverse the underlying disease mechanisms. Rhodiola crenulata extract (RCE) has been reported to alleviate AD symptoms in rats. However, its underlying mechanism of action is still unclear.Methods:A brain lipidomics study was conducted to investigate the protective effects of RCE against AD in rats to identify potential biomarkers of AD, Fourier-transform ion cyclotron resonance mass spectrometry (FT-ICR MS) coupled with high-performance reversed-phase liquid chromatography (RPLC) and hydrophilic interaction liquid chromatography (HILIC). Differences in lipid metabolism profiles were evaluated using multivariate statistical analysis. Finally, the possible mechanism of action of RCE on AD was investigate

In [150]:
#corpus to string
string = ' '.join(array_of_all_abstracts)

In [ ]:
#Making tokens from abstracts
tokens = word_tokenize(string)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Removing punctuation
words = [word for word in tokens if word.isalpha()]

In [ ]:
#Removing stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]

In [ ]:
#Printing top 10 occuring words
fdist = FreqDist(words)
top = fdist.most_common(10)
print("List of Removed Words:")
print(top)

List of Removed Words:
[('disease', 137), ('The', 112), ('PD', 96), ('patients', 90), ('AD', 81), ('neurodegenerative', 81), ('In', 75), ('study', 59), ('diseases', 58), ('protein', 54)]


In [ ]:
#new tokens without most frequently occuring 10 words
M = []

for w in words:
    if w not in top:
        M.append(w)

#removing repated words
cleansed = list(dict.fromkeys(M))
print(cleansed)

['Rationale', 'Alzheimer', 'disease', 'AD', 'chronic', 'severe', 'progressive', 'neurodegenerative', 'disorder', 'associated', 'cognitive', 'memory', 'impairment', 'ultimately', 'causes', 'death', 'Most', 'approved', 'drugs', 'alleviate', 'symptoms', 'interventions', 'found', 'reverse', 'underlying', 'mechanisms', 'Rhodiola', 'crenulata', 'extract', 'RCE', 'reported', 'rats', 'However', 'mechanism', 'action', 'still', 'A', 'brain', 'lipidomics', 'study', 'conducted', 'investigate', 'protective', 'effects', 'identify', 'potential', 'biomarkers', 'ion', 'cyclotron', 'resonance', 'mass', 'spectrometry', 'MS', 'coupled', 'liquid', 'chromatography', 'RPLC', 'hydrophilic', 'interaction', 'HILIC', 'Differences', 'lipid', 'metabolism', 'profiles', 'evaluated', 'using', 'multivariate', 'statistical', 'analysis', 'Finally', 'possible', 'investigated', 'analysing', 'metabolic', 'The', 'RPLCHILIC', 'results', 'showed', 'components', 'significant', 'differences', 'control', 'model', 'groups', 'Afte

In [ ]:
#Representing each abstract as a vector
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
K = vectorizer.fit(cleansed)
# summarize
print(K.vocabulary_)

# encode document
vector = vectorizer.transform(array_of_all_abstracts)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'rationale': 2643, 'alzheimer': 167, 'disease': 905, 'ad': 61, 'chronic': 509, 'severe': 2919, 'progressive': 2550, 'neurodegenerative': 2112, 'disorder': 908, 'associated': 275, 'cognitive': 551, 'memory': 1920, 'impairment': 1522, 'ultimately': 3350, 'causes': 459, 'death': 775, 'most': 2035, 'approved': 237, 'drugs': 957, 'alleviate': 144, 'symptoms': 3148, 'interventions': 1657, 'found': 1268, 'reverse': 2800, 'underlying': 3358, 'mechanisms': 1897, 'rhodiola': 2813, 'crenulata': 718, 'extract': 1181, 'rce': 2648, 'reported': 2743, 'rats': 2645, 'however': 1449, 'mechanism': 1896, 'action': 47, 'still': 3050, 'brain': 397, 'lipidomics': 1793, 'study': 3075, 'conducted': 623, 'investigate': 1673, 'protective': 2573, 'effects': 991, 'identify': 1491, 'potential': 2462, 'biomarkers': 377, 'ion': 1682, 'cyclotron': 743, 'resonance': 2768, 'mass': 1875, 'spectrometry': 3005, 'ms': 2051, 'coupled': 710, 'liquid': 1798, 'chromatography': 506, 'rplc': 2838, 'hydrophilic': 1468, 'interacti

In [174]:
#printing vectors of first 2 abstracts
vector.toarray()[0:2,:]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [172]:
#Clustering abstracts using Mini batch Kmeans
cls = MiniBatchKMeans(n_clusters=6, random_state=1)
cls.fit(vector)

print(cls.labels_)

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 1 0 1 0 0
 2 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 1 0 0 5]


In [164]:
def unique_words(cluster):
  #corpus to string
  string = ' '.join(cluster)

  #Making tokens from abstracts
  tokens = word_tokenize(string)

  #Removing punctuation
  words = [word for word in tokens if word.isalpha()]

  #Removing stopwords
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]

  #Printing 3 most unique words from each cluster
  from nltk.probability import FreqDist
  fdist = FreqDist(words)
  top = fdist.most_common()[-4:-1]
  print(top)

In [166]:
#Printing 3 unique words of each cluster

print("unique words in cluster0 :")
print(unique_words(cluster0))

print("unique words in cluster1 :")
print(unique_words(cluster1))

print("unique words in cluster2 :")
print(unique_words(cluster2))

print("unique words in cluster3 :")
print(unique_words(cluster3))

print("unique words in cluster4 :")
print(unique_words(cluster4))

print("unique words in cluster5 :")
print(unique_words(cluster5))


unique words in cluster0 :
[('hypotension', 1), ('potentials', 1), ('impactful', 1)]
None
unique words in cluster1 :
[('weakly', 1), ('change', 1), ('argues', 1)]
None
unique words in cluster2 :
[('neurons', 1), ('biological', 1), ('relevance', 1)]
None
unique words in cluster3 :
[('exists', 1), ('particular', 1), ('lead', 1)]
None
unique words in cluster4 :
[('possible', 1), ('determine', 1), ('specific', 1)]
None
unique words in cluster5 :
[('plausible', 1), ('mechanism', 1), ('underlying', 1)]
None
